## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

# The applied options are for CSV files. For other file types, these will be ignored.
df =spark.read.csv(file_location,header=True,inferSchema=True)
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
df.columns

['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

### Handling Categorical Features

In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
indexer=StringIndexer(inputCol="sex",outputCol="sex_indexed")
df_r=indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|        0.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|        0.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|        0.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|        1.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        0.0|
|     18.43| 3.0|  Male|    No|Sun|Dinne

In [0]:
indexer=StringIndexer(inputCols=["smoker","day","time"],
                      outputCols=["smoker_indexed","day_indexed","time_index"])
df_r=indexer.fit(df_r).transform(df_r)
df_r.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_index|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|       0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|       0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|       0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|       0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|       0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|       0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|       0.0|
|     26.88|3.12|  M

In [0]:
df_r.columns

['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'time_index']

### Group inputCols and outputCol using `VectorAssembler` into one featureassembler

In [0]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['total_bill','size','sex_indexed','smoker_indexed','day_indexed',
                          'time_index'],outputCol="Independent Features")
output=featureassembler.transform(df_r)

In [0]:
output.select('Independent Features').show()

+--------------------+
|Independent Features|
+--------------------+
|[16.99,2.0,1.0,0....|
|[10.34,3.0,0.0,0....|
|[21.01,3.0,0.0,0....|
|[23.68,2.0,0.0,0....|
|[24.59,4.0,1.0,0....|
|[25.29,4.0,0.0,0....|
|[8.77,2.0,0.0,0.0...|
|[26.88,4.0,0.0,0....|
|[15.04,2.0,0.0,0....|
|[14.78,2.0,0.0,0....|
|[10.27,2.0,0.0,0....|
|[35.26,4.0,1.0,0....|
|[15.42,2.0,0.0,0....|
|[18.43,4.0,0.0,0....|
|[14.83,2.0,1.0,0....|
|[21.58,2.0,0.0,0....|
|[10.33,3.0,1.0,0....|
|[16.29,3.0,0.0,0....|
|[16.97,3.0,1.0,0....|
|(6,[0,1],[20.65,3...|
+--------------------+
only showing top 20 rows



In [0]:
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+----------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_index|Independent Features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+----------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|       0.0|[16.99,2.0,1.0,0....|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|       0.0|[10.34,3.0,0.0,0....|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|       0.0|[21.01,3.0,0.0,0....|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|       0.0|[23.68,2.0,0.0,0....|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|       0.0|[24.59,4.0,1.0,0....|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| 

### Finilize data

In [0]:
finalized_data=output.select("Independent Features","tip")
finalized_data.show()

+--------------------+----+
|Independent Features| tip|
+--------------------+----+
|[16.99,2.0,1.0,0....|1.01|
|[10.34,3.0,0.0,0....|1.66|
|[21.01,3.0,0.0,0....| 3.5|
|[23.68,2.0,0.0,0....|3.31|
|[24.59,4.0,1.0,0....|3.61|
|[25.29,4.0,0.0,0....|4.71|
|[8.77,2.0,0.0,0.0...| 2.0|
|[26.88,4.0,0.0,0....|3.12|
|[15.04,2.0,0.0,0....|1.96|
|[14.78,2.0,0.0,0....|3.23|
|[10.27,2.0,0.0,0....|1.71|
|[35.26,4.0,1.0,0....| 5.0|
|[15.42,2.0,0.0,0....|1.57|
|[18.43,4.0,0.0,0....| 3.0|
|[14.83,2.0,1.0,0....|3.02|
|[21.58,2.0,0.0,0....|3.92|
|[10.33,3.0,1.0,0....|1.67|
|[16.29,3.0,0.0,0....|3.71|
|[16.97,3.0,1.0,0....| 3.5|
|(6,[0,1],[20.65,3...|3.35|
+--------------------+----+
only showing top 20 rows



### LinearRegression

In [0]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='tip')
regressor=regressor.fit(train_data)

In [0]:
regressor.coefficients

DenseVector([0.1004, 0.1265, 0.06, -0.0965, 0.0358, -0.0987])

In [0]:
regressor.intercept

0.7274902733950028

In [0]:
### Predictions
pred_results=regressor.evaluate(test_data)

In [0]:
## Final comparison
pred_results.predictions.show()

+--------------------+----+------------------+
|Independent Features| tip|        prediction|
+--------------------+----+------------------+
|(6,[0,1],[10.51,2...|1.25|2.0351900837472447|
|(6,[0,1],[12.02,2...|1.97|2.1867257396649302|
|(6,[0,1],[13.28,2...|2.72|2.3131727108280318|
|(6,[0,1],[13.37,2...| 2.0| 2.322204637339682|
|(6,[0,1],[17.81,4...|2.34| 3.020751181850633|
|(6,[0,1],[19.82,2...|3.18|2.9694927040079406|
|(6,[0,1],[20.65,3...|3.35|3.1792728890279305|
|(6,[0,1],[31.27,3...| 5.0| 4.245040217402645|
|[3.07,1.0,1.0,1.0...| 1.0|1.1256380473546135|
|[9.68,2.0,0.0,0.0...|1.32|1.9876662514598449|
|[9.94,2.0,0.0,0.0...|1.56|2.0137584836046116|
|[10.27,2.0,0.0,0....|1.71|2.0468755474806626|
|[10.29,2.0,1.0,0....| 2.6| 2.108919542151285|
|[10.34,3.0,0.0,0....|1.66| 2.180386130846718|
|[10.65,2.0,1.0,0....| 1.5|2.0821620420876767|
|[11.59,2.0,0.0,1....| 1.5| 2.047109275535341|
|[12.16,2.0,0.0,1....| 2.2| 2.112967472861218|
|[12.46,2.0,0.0,0....| 1.5| 2.338193628126451|
|[12.74,2.0,1

In [0]:
### PErformance Metrics
pred_results.r2,pred_results.meanAbsoluteError,pred_results.meanSquaredError

(0.4416536422452384, 0.7529922640406421, 0.9582455693495309)